In [1]:
import os
import pickle
import csv
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow

In [9]:
def get_authenticated_service():
    credentials = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            credentials = pickle.load(token)
    #  Check if the credentials are invalid or do not exist
    if not credentials or not credentials.valid:
        # Check if the credentials have expired
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                CLIENT_SECRETS_FILE, SCOPES)
            credentials = flow.run_console()

        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(credentials, token)

    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)


def write_to_csv(comments, fileTitle):
    with open('data\\%s.csv' % fileTitle, 'w', encoding="utf-8-sig") as comments_file:
        comments_writer = csv.writer(comments_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

        comments_writer.writerow(['Comment','likeCount'])
        print('begin to write video %s', fileTitle)
        for row in comments:
            # convert the tuple to a list and write to the output file
            comments_writer.writerow(list(row))
            

In [26]:
SCOPES = ["https://www.googleapis.com/auth/youtube.force-ssl"]
CLIENT_SECRETS_FILE = "client_secret_675337332293-6jel2e9kdgendpl20q9s23d91c9lkb9r.apps.googleusercontent.com.json"
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
youtube = get_authenticated_service()


with open('video.info', 'rb') as file: # video info list
    videos = pickle.load(file) 

In [52]:
for i in range(0, len(videos)):
    videoId = videos[i]['snippet']['resourceId']['videoId']
    videoTitle = videos[i]['snippet']['title']
    comments = []
    
    try:
        request = youtube.commentThreads().list(part="snippet,replies", order="time", videoId=videoId, textFormat='plainText').execute() #in case of the comments disable
    except:
        continue  #comment disable will raise a HTTPError 403

    while request:
        for item in request['items']:
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay'] #comments
            likeCountComment = item['snippet']['topLevelComment']['snippet']['likeCount'] #like count
            comments.append([comment,likeCountComment])
 
        # Check if next page exists
        if 'nextPageToken' in request:

            request = youtube.commentThreads().list(part="snippet,replies", order="time", videoId=videoId, textFormat='plainText', pageToken=request['nextPageToken']).execute()
        else:
            break
    
    if comments:
        write_to_csv(comments, videoTitle)  
    

begin to write video %s 鄂东老男孩的YouTube帐号原创作者声明
begin to write video %s 老男孩：上门给老乡修洗衣机，大婶说老男孩以前瘦些，现在长胖了
begin to write video %s 老男孩的倒三轮出了问题，售后寄来了配件，老男孩自己当修车工
begin to write video %s 这台液晶电视放着会黑屏，老男孩没拆零件就说修好了，这样靠谱吗
begin to write video %s 老男孩上门维修，大叔对老男孩的这车子感兴趣，上来就问多少钱
begin to write video %s 老男孩和家人去五脑山看菊花，遇到好玩的，忍不住也来皮一下
begin to write video %s 上次清蒸蟹吃完不过瘾，老男孩这次又做1大盆香辣蟹，秒变吃货
begin to write video %s 大叔家电视坏了，老男孩上门去维修，大婶说自家人应该来吃饭
begin to write video %s 网友带来1网兜大闸蟹，老男孩又有口福了，清蒸还是香辣的好吃呢
begin to write video %s 隔壁村部的音响坏了，老男孩去维修，什么情况竟当场飙起了英语
begin to write video %s 老男孩上门去维修，座驾刚停稳就被人看上了，不仅霸座还要拍照
begin to write video %s 老男孩骑着三轮去维修，什么问题不需要工具，还说没遇到这种怪事
begin to write video %s 老男孩给自己放个假，跟堂弟一起去看冰臼群，又遇到了认识的网友
begin to write video %s 镇里学校举行国庆70周年活动，老男孩又1次扩展业务，担任摄影师
begin to write video %s 老男孩上门修电视，刚进门试机却发现遇到大麻烦，还好运气不错
begin to write video %s 黄冈市第2届一节一会，老男孩也去凑热闹，进展会就开始蹭吃蹭喝
begin to write video %s 老男孩父子和大伯去姑奶家，还修好1台老电视，表婶做了1桌好菜
begin to write video %s 大婶家的老电视坏了，老男孩极力劝说不要修，大叔却又搬来1台
begin to write video %s 网友老乡送